In [1]:
import os
import sys
import socket
from requests import get
from pathlib import Path


IS_LOCAL = 'NB_ENV' in os.environ

if IS_LOCAL: #local
    from my_utils import get_notebook_path
    
    ROOT_DIR = Path('../../')
    NB = get_notebook_path().split('/')[-1].split('.')[0]
    MLFLOW_URL = 'http://192.168.1.99:9000'
    PATH_SEP = '\\'
    HOST = socket.gethostname()
else: # colab
    from google.colab import drive
    drive.mount('/content/drive')
    
    !pip install mlflow optuna logzero > /dev/null
    !nvidia-smi
    !cat/proc/uptime|awk'{print $1 /60 /60 "hours"}'
    
    ROOT_DIR = Path('/content/drive/MyDrive/ml/Indoor')
    NB = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'].split('.')[0]
    MLFLOW_URL = 'http://yururoi.f5.si:9000'
    PATH_SEP = '/'
    HOST = 'COLAB'
    
    sys.path.append('/content/drive/MyDrive/ml/Indoor/src')

DESCRIPTION='eda'
NB, HOST, DESCRIPTION

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tue Mar 23 05:59:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

('exp003-0', 'COLAB', 'eda')

In [2]:
import warnings
import os
import re
import json
import itertools

import mlflow
from dataclasses import dataclass
from dataclasses import asdict

import pandas as pd
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, average_precision_score, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

from logzero import logger

from my_utils import seed_torch
from my_utils import to_pickle
from my_utils import unpickle

import pickle
import glob
from io_f import read_data_file

from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

from collections import OrderedDict

tqdm.pandas()

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)


DATA_DIR = ROOT_DIR / Path('data')


test_site_ids = ['5da138754db8ce0c98bca82f', '5dc8cea7659e181adb076a3f', '5da138764db8ce0c98bcaa46', '5d2709bb03f801723c32852c', '5dbc1d84c1eb61796cf7c010', '5da1389e4db8ce0c98bd0547', '5d27096c03f801723c31e5e0', '5d2709c303f801723c3299ee', '5da138364db8ce0c98bc00f1', '5d27075f03f801723c2e360f', '5da1383b4db8ce0c98bc11ab', '5d2709a003f801723c3251bf', '5d27097f03f801723c320d97', '5d2709b303f801723c327472', '5da958dd46f8266d0737457b', '5da1382d4db8ce0c98bbe92e', '5d27099f03f801723c32511d', '5a0546857ecc773753327266', '5da138b74db8ce0c98bd4774', '5c3c44b80379370013e0fd2b', '5da138314db8ce0c98bbf3a0', '5d2709e003f801723c32d896', '5da138274db8ce0c98bbd3d2', '5d2709d403f801723c32bd39']

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
class Config:
    N_LABEL = 24
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 0.001
    PATIENCE = 10
    EPOCH = 100
    BATCH_SIZE = 1024
    N_REPEATS = 3
    SKIP_EVALUATE_NUM = 5
    NUM_FEATS = 20

seed_torch(seed=Config.RANDOM_SATE)

In [4]:
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]


In [5]:
train_files = sorted(glob.glob(str(DATA_DIR / 'wifi/*_train.csv')))
test_files = sorted(glob.glob(str(DATA_DIR / 'wifi/*_test.csv')))
sample_submission = pd.read_csv(DATA_DIR / 'sample_submission.csv', index_col=0)

In [6]:
with open(DATA_DIR / 'wifi/train_all.pkl', 'rb') as f:
  data = pickle.load( f)

with open(DATA_DIR / 'wifi/test_all.pkl', 'rb') as f:
  test_data = pickle.load(f)

In [7]:
data

,bssid_0,bssid_1,bssid_2,bssid_3,bssid_4,bssid_5,bssid_6,bssid_7,bssid_8,bssid_9,bssid_10,bssid_11,bssid_12,bssid_13,bssid_14,bssid_15,bssid_16,bssid_17,bssid_18,bssid_19,bssid_20,bssid_21,bssid_22,bssid_23,bssid_24,bssid_25,bssid_26,bssid_27,bssid_28,bssid_29,bssid_30,bssid_31,bssid_32,bssid_33,bssid_34,bssid_35,bssid_36,bssid_37,bssid_38,bssid_39,bssid_40,bssid_41,bssid_42,bssid_43,bssid_44,bssid_45,bssid_46,bssid_47,bssid_48,bssid_49,bssid_50,bssid_51,bssid_52,bssid_53,bssid_54,bssid_55,bssid_56,bssid_57,bssid_58,bssid_59,bssid_60,bssid_61,bssid_62,bssid_63,bssid_64,bssid_65,bssid_66,bssid_67,bssid_68,bssid_69,bssid_70,bssid_71,bssid_72,bssid_73,bssid_74,bssid_75,bssid_76,bssid_77,bssid_78,bssid_79,bssid_80,bssid_81,bssid_82,bssid_83,bssid_84,bssid_85,bssid_86,bssid_87,bssid_88,bssid_89,bssid_90,bssid_91,bssid_92,bssid_93,bssid_94,bssid_95,bssid_96,bssid_97,bssid_98,bssid_99,rssi_0,rssi_1,rssi_2,rssi_3,rssi_4,rssi_5,rssi_6,rssi_7,rssi_8,rssi_9,rssi_10,rssi_11,rssi_12,rssi_13,rssi_14,rssi_15,rssi_16,rssi_17,rssi_18,rssi_19,rssi_20,rssi_21,rssi_22,rssi_23,rssi_24,rssi_25,rssi_26,rssi_27,rssi_28,rssi_29,rssi_30,rssi_31,rssi_32,rssi_33,rssi_34,rssi_35,rssi_36,rssi_37,rssi_38,rssi_39,rssi_40,rssi_41,rssi_42,rssi_43,rssi_44,rssi_45,rssi_46,rssi_47,rssi_48,rssi_49,rssi_50,rssi_51,rssi_52,rssi_53,rssi_54,rssi_55,rssi_56,rssi_57,rssi_58,rssi_59,rssi_60,rssi_61,rssi_62,rssi_63,rssi_64,rssi_65,rssi_66,rssi_67,rssi_68,rssi_69,rssi_70,rssi_71,rssi_72,rssi_73,rssi_74,rssi_75,rssi_76,rssi_77,rssi_78,rssi_79,rssi_80,rssi_81,rssi_82,rssi_83,rssi_84,rssi_85,rssi_86,rssi_87,rssi_88,rssi_89,rssi_90,rssi_91,rssi_92,rssi_93,rssi_94,rssi_95,rssi_96,rssi_97,rssi_98,rssi_99,x,y,floor,path,site_id
0,db01605eac3f33540038bd9722aba25774871d43,965f254a2e8d05bbb40bd2413ff61de3ad6c4151,0b64e537cc3d1818ec46f94f8dc14043a98d0089,922e582c66016a2b9f64e38f89ebe82f66eefb24,dc4c46287575c45f3e32c022d868d047b485ed4c,93e20595eeef175d3aa3c3381f6a22ee792d48d9,b2b0ddbb5a2aadfc6ab2f388db584b6c280d3f82,8c936564ea4b4300576f53136505527eb5972c07,61c3aaf1a526f808c05952ea3f098e37354a674a,3f564032c7eebc173b38aee35225e323d4389faf,2ed2e7ff03b520f03de97026028d9aada89dd4ba,0f5daed11a61e0d6941a1a42ff428ca216d61003,e315aa306303cc8df91c60dcbd80145ef496b649,1f37bbb3f42125f665b83584d0376b21ec3eb43c,c8f3983a40e3c462cdfd155f3d0c77868a717ffd,4c83a7a1e51bfa8a5fa20e854ab3feec057c52c9,d090480c29bbcafe61fdf7bf491e23c606d3f181,599fa96d549ed870671d6bc1927aaa8bbaacca12,51782c2fabefa97e99dca895fd36f1a47e214610,db85cb1371261046f2b711ac8dcceeb06b7ca724,53a65f2ceb68d4fab8fe912e36b2ca5d410a61be,e57024faa1bfe6de5d71e1b779bff11e72e97e1d,71d97fb3d6f464d4c26fd061732e58398d053b2e,add8b64585688391c11f2e0d1ca9039b70c802e5,ce28608c3d091ac0d25d84459ebad253edf83e1f,edbe427e90919b00d712ceb8c1ee8e92a54051e2,1bb0e992cff45a54d29e97f47a7d1281435a5e3b,172b8a530c6f1946a495576ea9e885874db2f288,1b76d774360c4cfaf89e75ec3a3add7c5201b885,ca86c5b074c5768e481e069b751bf22c6d95bd48,40d99a3e5214aa704f637b7d72631e69550ee256,46c934893439700099d03a6892ea934ecb2729d6,fe3211f90e4ab1f500e10fe175ae6142f4b13130,569878b7f574a738d49e365f45c49f44b81936f2,9f6279762e596eb60cd9a3870c1c305692d8e338,ae54f8552a572ddf81302b56c07f63c6321270be,3c89886dd08bb4a24384cf8bc0c6423e4804e6e9,5c10b343d767a30515e6015de25751a2883328f8,16374260af7d03b10f167358a4f6a70620e131f4,f26678bbbbd078e242638a0d1fb5ba2e61262f4c,4e09a5f7eb9df06543b1dd015b272cae5f69b48d,b710e05e90e027ac2d0bc5a5fc8fa06d4a2bab2e,08f7de5409dd4c4df92a9be0e8daebc871d3e7d1,b51107fb094a5127ad76ef502ac03632a8f0d67c,c34f73e48a2289918d2e8cf179771e8ee8cd1cfa,2db93d0350ecfc751c14d2d349dd8d2bbd8007e0,33f83c3b2dbba620b413afa1af33114986298012,5024a162dfac11512ba8cb59eccd836abe70cf42,e32eb454533b38ef2da6866f47110a680f8a37f7,284be046617451d2c3450c11ebf5a5c45a84ac90,cf3425314e459d907136d918566a069aba76247e,405fa41dd78d5f5182b0e3f8832eda69dc0a49d7,62f63715ba99871e176526ffe52ad2be5d200688,f53a8fec6a3a416fa9972e9e8b6297480c6452d5,d8b4b0bebc252703de74c2effaf574bc4d6140f0,c3423704f962271dae5b6081d54322dddd02fc8

In [8]:
BSSID_FEATS = [f'bssid_{i}' for i in range(Config.NUM_FEATS)]
RSSI_FEATS  = [f'rssi_{i}' for i in range(Config.NUM_FEATS)]

In [9]:
# bssidの一覧作成
# wifi_bassidにはtrainとtest両方のbssidの一覧が含まれる

wifi_bssids = []
for i in range(100):
    wifi_bssids.extend(data.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids_test = []
for i in range(100):
    wifi_bssids_test.extend(test_data.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids.extend(wifi_bssids_test)
wifi_bssids_size = len(wifi_bssids)

BSSID TYPES: 61206
BSSID TYPES: 33042


## LabelEncoding & 標準化

In [10]:
# Label Encoding

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(data['site_id'])

ss = StandardScaler()
ss.fit(data.loc[:,RSSI_FEATS])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
data.loc[:,RSSI_FEATS] = ss.transform(data.loc[:,RSSI_FEATS])
for i in BSSID_FEATS:
    data.loc[:,i] = le.transform(data.loc[:,i])
    data.loc[:,i] = data.loc[:,i] + 1
    
data.loc[:, 'site_id'] = le_site.transform(data.loc[:, 'site_id'])

data.loc[:,RSSI_FEATS] = ss.transform(data.loc[:,RSSI_FEATS])

In [12]:
test_data.loc[:,RSSI_FEATS] = ss.transform(test_data.loc[:,RSSI_FEATS])
for i in BSSID_FEATS:
    test_data.loc[:,i] = le.transform(test_data.loc[:,i])
    test_data.loc[:,i] = test_data.loc[:,i] + 1
    
test_data.loc[:, 'site_id'] = le_site.transform(test_data.loc[:, 'site_id'])

test_data.loc[:,RSSI_FEATS] = ss.transform(test_data.loc[:,RSSI_FEATS])


In [13]:
site_count = len(data['site_id'].unique())
data.reset_index(drop=True, inplace=True)

In [14]:

class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve after a given patience.
    based on: https://github.com/Bjarten/early-stopping-pytorch
    """
    def __init__(self, patience=7, verbose=False, delta=0, fold=''):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            logger.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            logger.info(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            
        torch.save(model.state_dict(), ROOT_DIR / Path('output') / f'checkpoint_{NB}_{fold}.pt')
        self.val_loss_min = val_loss

In [15]:
class IndoorWiFiNet(nn.Module):
    def __init__(self, bssid_size, site_size):
        super(IndoorWiFiNet, self).__init__()


        self.bssid_embedding = nn.Embedding(bssid_size, 64, max_norm=True)
        
        self.site_embedding = nn.Embedding(site_size, 64, max_norm=True)

        self.rssi = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.Linear(20, Config.NUM_FEATS * 64)
        )


        concat_size = 64 + (Config.NUM_FEATS * 64) + (Config.NUM_FEATS * 64)
        self.bn1 = nn.BatchNorm1d(concat_size)

        self.flatten = nn.Flatten()

        self.dropout1 = nn.Dropout(0.3)
        self.linear1 = nn.Linear(in_features=concat_size, out_features=256)#, bias=False)
        self.bn2 = nn.BatchNorm1d(256)

        self.linear2 = nn.Linear(in_features=256, out_features=128)#, bias=False)
        self.linear3 = nn.Linear(in_features=128, out_features=16)#, bias=False)

        self.bn3 = nn.BatchNorm1d(128)
        self.bn4 = nn.BatchNorm1d(16)


        self.xy = nn.Linear(in_features=16, out_features=2)#, bias=False)
        self.floor = nn.Linear(in_features=16, out_features=1)#, bias=False)
        

    def forward(self, bssid, rssi, site):

        site = torch.reshape(site, (-1, 1))

        bssid_out = self.bssid_embedding(bssid)
        site_out = self.site_embedding(site)

        rssi_out = self.rssi(rssi)


        bssid_out = self.flatten(bssid_out)
        site_out = self.flatten(site_out)

        x = torch.cat([bssid_out, rssi_out, site_out], dim=1)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = F.relu(self.linear1(x))
        x = self.bn2(x)

        x = F.relu(self.linear2(x))
        x = self.bn3(x)

        x = F.relu(self.linear3(x))
        x = self.bn4(x)

        #x = self.lstm(x)

        xy = self.xy(x)
        floor = self.floor(x)

        return xy, floor

In [24]:
class IndoorWiFiDataSet(Dataset):
    def __init__(self, wifi_df, data_type='test'):
        self.wifi_df = wifi_df
        self.data_type = data_type
        self.bssids = wifi_df[BSSID_FEATS].to_numpy().astype(np.int)
        self.rssis = wifi_df[RSSI_FEATS].to_numpy().astype(np.float)
        self.site_ids = wifi_df['site_id'].to_numpy()
        if data_type in ['train', 'valid']:
            self.floors = wifi_df['floor'].to_numpy().astype(np.int)
            self.xs = wifi_df['x'].to_numpy()
            self.ys = wifi_df['y'].to_numpy()
        
    def __len__(self):
        return len(self.wifi_df)

    def __getitem__(self, idx):

        #wifi_row = self.wifi_df.iloc[idx]

        bssid = self.bssids[idx,:]
        rssi = self.rssis[idx,:]
        site_id = self.site_ids[idx]

        if self.data_type in ['test', 'valid']:
            return bssid, rssi, site_id

        elif self.data_type == 'train':
            x = self.xs[idx]
            y = self.ys[idx]
            floor = self.floors[idx]

            return bssid, rssi, site_id, x, y, floor


In [17]:
"""class IndoorWiFiDataSet(Dataset):
    def __init__(self, wifi_df, data_type='test'):
        self.wifi_df = wifi_df
        self.data_type = data_type
        self.bssids = data[BSSID_FEATS].to_numpy().astype(np.int)
        self.rssis = data[RSSI_FEATS].to_numpy().astype(np.float)
        self.floors = data['floor'].to_numpy().astype(np.int)
        self.site_ids = data['site_id'].to_numpy()
        self.xs = data['x'].to_numpy()
        self.ys = data['y'].to_numpy()
        
    def __len__(self):
        return len(self.wifi_df)

    def __getitem__(self, idx):

        wifi_row = self.wifi_df.iloc[idx]

        bssid = self.bssids[idx,:]
        rssi = self.rssis[idx,:]
        site_id = self.site_ids[idx]

        if self.data_type in ['test', 'valid']:
            return bssid, rssi, site_id

        elif self.data_type == 'train':
            x = wifi_row['x']
            y = wifi_row['y']
            floor = self.floors[idx]

            return bssid, rssi, site_id, x, y, floor
"""

"class IndoorWiFiDataSet(Dataset):\n    def __init__(self, wifi_df, data_type='test'):\n        self.wifi_df = wifi_df\n        self.data_type = data_type\n        self.bssids = data[BSSID_FEATS].to_numpy().astype(np.int)\n        self.rssis = data[RSSI_FEATS].to_numpy().astype(np.float)\n        self.floors = data['floor'].to_numpy().astype(np.int)\n        self.site_ids = data['site_id'].to_numpy()\n        self.xs = data['x'].to_numpy()\n        self.ys = data['y'].to_numpy()\n        \n    def __len__(self):\n        return len(self.wifi_df)\n\n    def __getitem__(self, idx):\n\n        wifi_row = self.wifi_df.iloc[idx]\n\n        bssid = self.bssids[idx,:]\n        rssi = self.rssis[idx,:]\n        site_id = self.site_ids[idx]\n\n        if self.data_type in ['test', 'valid']:\n            return bssid, rssi, site_id\n\n        elif self.data_type == 'train':\n            x = wifi_row['x']\n            y = wifi_row['y']\n            floor = self.floors[idx]\n\n            return b

In [18]:
"""class IndoorWiFiDataSet(Dataset):
    def __init__(self, wifi_df, data_type='test'):
        self.wifi_df = wifi_df
        self.data_type = data_type
        
    def __len__(self):
        return len(self.wifi_df)

    def __getitem__(self, idx):

        wifi_row = self.wifi_df.iloc[idx]

        bssid = wifi_row[BSSID_FEATS].to_numpy().astype(np.int)
        rssi = wifi_row[RSSI_FEATS].to_numpy().astype(np.float)
        site_id = wifi_row['site_id']

        if self.data_type in ['test', 'valid']:
            return bssid, rssi, site_id

        elif self.data_type == 'train':
            x = wifi_row['x']
            y = wifi_row['y']
            floor = wifi_row['floor'].astype(np.int)

            return bssid, rssi, site_id, x, y, floor"""


"class IndoorWiFiDataSet(Dataset):\n    def __init__(self, wifi_df, data_type='test'):\n        self.wifi_df = wifi_df\n        self.data_type = data_type\n        \n    def __len__(self):\n        return len(self.wifi_df)\n\n    def __getitem__(self, idx):\n\n        wifi_row = self.wifi_df.iloc[idx]\n\n        bssid = wifi_row[BSSID_FEATS].to_numpy().astype(np.int)\n        rssi = wifi_row[RSSI_FEATS].to_numpy().astype(np.float)\n        site_id = wifi_row['site_id']\n\n        if self.data_type in ['test', 'valid']:\n            return bssid, rssi, site_id\n\n        elif self.data_type == 'train':\n            x = wifi_row['x']\n            y = wifi_row['y']\n            floor = wifi_row['floor'].astype(np.int)\n\n            return bssid, rssi, site_id, x, y, floor"

In [19]:
class IndoorLoss(nn.Module):
    def __init__(self):
        super().__init__()

        self.mse = nn.MSELoss()

    def forward(self, x, y, floor, x_pred, y_pred, floor_pred):

        x_loss = self.mse(x, x_pred)
        y_loss = self.mse(y, y_pred)
        floor_loss = self.mse(floor, floor_pred)

        return x_loss + y_loss + floor_loss

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
data.shape

(258125, 205)

In [22]:
%%time

oof_x = np.zeros((len(data)))
oof_y = np.zeros((len(data)))
oof_floor = np.zeros((len(data)))
cv_scores = {}



kf = KFold(n_splits=Config.N_FOLD, random_state=42, shuffle=True)
for fold, (train_index, valid_index) in enumerate(kf.split(data)):
    
    print(f'\r\n====== {fold} ======')
    
    net = IndoorWiFiNet(wifi_bssids_size, site_count)
    net = net.to(device)

    #criterion = IndoorLoss()
    criterion = nn.MSELoss()
    criterion = criterion.to(device)
    #optimizer = optim.SGD(net.parameters(), lr=Config.LR, weight_decay=0.0001, momentum=0.9)
    optimizer = torch.optim.Adam(net.parameters(), lr=Config.LR)
    
    train, valid = data.iloc[train_index], data.iloc[valid_index]

    # TODO oof用のdataloaderはいらないはず
    train_dataset = IndoorWiFiDataSet(train.reset_index(drop=True), data_type='train')
    valid_dataset = IndoorWiFiDataSet(valid.reset_index(drop=True), data_type='valid')

    trainloader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True, drop_last=True, num_workers=4)
    validloader = DataLoader(valid_dataset, batch_size=Config.BATCH_SIZE ,num_workers=4)

    early_stopping = EarlyStopping(patience=Config.PATIENCE, verbose=True, fold=fold)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.EPOCH)


    ############# TRAIN #############
    val_losses = []
    val_metrics = []
    for epoch in range(Config.EPOCH):

        running_loss = 0.0
        #train_lwlrap = 0.0
        n_iter = len(trainloader)
        with tqdm(enumerate(trainloader), total=n_iter) as pbar:
            for i, (bssid, rssi, site_id, x, y, floor) in pbar:
                net.train()
                # zero the parameter gradients
                optimizer.zero_grad()

                bssid, rssi, site_id, x, y, floor = bssid.to(device).long(), rssi.to(device).float(), site_id.to(device).long(), x.to(device).float(), y.to(device).float(), floor.to(device)

                xy_pred, floor_pred = net(bssid, rssi, site_id)
                xy_pred = xy_pred.transpose(0, 1).squeeze(-2).reshape(-1)
                label = torch.cat([x, y], dim=-1)
                loss = criterion(xy_pred, label)
                #loss = criterion(x, y, floor, x_pred, y_pred, floor_pred)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                pbar.set_postfix(OrderedDict(
                    epoch="{:>10}".format(epoch), loss="{:.4f}".format(loss.item())
                ))
            #scheduler.step()
            
        #if epoch < Config.SKIP_EVALUATE_NUM:
        #    continue

        ############# VALID #############
        val_loss = 0.0
        val_lwlrap = 0.0
        x_preds = np.array([])
        y_preds = np.array([])
        val_preds_frame = []
        n_iter_val = len(validloader)
        for i, (bssid, rssi, site_id) in tqdm(enumerate(validloader), total=len(validloader)):
            net.eval()

            with torch.no_grad():
                
                #melspec, labels = melspec.to(device).float(), labels.to(device).float()
                bssid, rssi, site_id = bssid.to(device).long(), rssi.to(device).float(), site_id.to(device).long()
                xy_pred, floor_pred = net(bssid, rssi, site_id)

                xy_pred = xy_pred.to('cpu').detach().numpy().copy()
                x_pred, y_pred = np.hsplit(xy_pred, 2)
                x_preds = np.concatenate([x_preds, x_pred.reshape(-1)])
                y_preds = np.concatenate([y_preds, y_pred.reshape(-1)])

        score = comp_metric(x_preds, y_preds, 0, valid['x'], valid['y'], 0)

        early_stopping(score, net)

        if early_stopping.early_stop:
            logger.info("Early stopping")
            cv_scores[f'cv{fold}'] = early_stopping.best_score
            break
                
    _x_oof = np.array([])
    _y_oof = np.array([])
    ############# OOF #############
    oof_preds = []
    for i, (bssid, rssi, site_id) in tqdm(enumerate(validloader), total=len(validloader)):
        net.load_state_dict(torch.load(ROOT_DIR / Path('output') / f'checkpoint_{NB}_{fold}.pt'))
        net.eval()

        with torch.no_grad():

            bssid, rssi, site_id, x, y, floor = bssid.to(device).long(), rssi.to(device).float(), site_id.to(device).long(), x.to(device).float(), y.to(device).float(), floor.to(device)

            xy_pred, floor_pred = net(bssid, rssi, site_id)

            xy_pred = xy_pred.to('cpu').detach().numpy().copy()
            x_pred, y_pred = np.hsplit(xy_pred, 2)
            _x_oof = np.concatenate([_x_oof, x_pred.reshape(-1)])
            _y_oof = np.concatenate([_y_oof, y_pred.reshape(-1)])

    print(comp_metric(_x_oof, _y_oof, 0, valid['x'], valid['y'], 0))

    oof_x[valid_index] = _x_oof
    oof_y[valid_index] = _y_oof

    del net
    torch.cuda.empty_cache()
    


====== 0 ======


100%|██████████| 51/51 [00:00<00:00, 88.09it/s] 
[I 210323 05:59:57 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (inf --> 164.623394).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 84.53it/s] 
[I 210323 06:00:01 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (164.623394 --> 159.801722).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 81.93it/s]
[I 210323 06:00:06 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (159.801722 --> 149.688327).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 77.83it/s]
[I 210323 06:00:11 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (149.688327 --> 139.414402).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 66.41it/s]
[I 210323 06:00:16 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (139.414402 --> 124.581857).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 52.02it/s]
[I 210323 06:00:21 <ipython-input-14-fa517e1eef50>:45] Validation loss decrea

5.661452296704021

====== 1 ======


100%|██████████| 51/51 [00:00<00:00, 68.94it/s]
[I 210323 06:08:01 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (inf --> 164.644656).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 81.48it/s]
[I 210323 06:08:06 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (164.644656 --> 159.213041).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 79.48it/s]
[I 210323 06:08:10 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (159.213041 --> 150.280401).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 78.09it/s]
[I 210323 06:08:15 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (150.280401 --> 138.923219).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 83.14it/s]
[I 210323 06:08:20 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (138.923219 --> 125.323639).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 84.27it/s]
[I 210323 06:08:24 <ipython-input-14-fa517e1eef50>:45] Validation loss decrease

5.629378711411546

====== 2 ======


...

5.694631717897945

====== 3 ======


100%|██████████| 51/51 [00:00<00:00, 123.01it/s]
[I 210323 06:24:19 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (inf --> 163.726863).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 121.38it/s]
[I 210323 06:24:23 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (163.726863 --> 158.896364).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 124.30it/s]
[I 210323 06:24:27 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (158.896364 --> 149.531918).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 125.96it/s]
[I 210323 06:24:32 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (149.531918 --> 138.462897).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 122.24it/s]
[I 210323 06:24:36 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (138.462897 --> 124.253313).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 124.31it/s]
[I 210323 06:24:40 <ipython-input-14-fa517e1eef50>:45] Validation loss de

5.912216989754758

====== 4 ======


100%|██████████| 51/51 [00:00<00:00, 116.49it/s]
[I 210323 06:28:19 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (inf --> 164.478227).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 117.28it/s]
[I 210323 06:28:23 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (164.478227 --> 159.476179).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 111.08it/s]
[I 210323 06:28:28 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (159.476179 --> 149.944667).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 118.46it/s]
[I 210323 06:28:32 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (149.944667 --> 138.227058).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 116.57it/s]
[I 210323 06:28:37 <ipython-input-14-fa517e1eef50>:45] Validation loss decreased (138.227058 --> 124.982643).  Saving model ...
100%|██████████| 51/51 [00:00<00:00, 115.96it/s]
[I 210323 06:28:41 <ipython-input-14-fa517e1eef50>:45] Validation loss de

5.767399172113271
CPU times: user 18min 28s, sys: 9min 5s, total: 27min 34s
Wall time: 35min


In [25]:
%%time

x_preds_mean = np.zeros(len(test_data))
y_preds_mean = np.zeros(len(test_data))

for fold in range(Config.N_FOLD):

    x_preds = np.array([])
    y_preds = np.array([])

    net = IndoorWiFiNet(wifi_bssids_size, site_count)
    net.to(device)
    print(ROOT_DIR / Path('output') / f'load checkpoint_{NB}_{fold}.pt')
    net.load_state_dict(torch.load(ROOT_DIR / Path('output') / f'checkpoint_{NB}_{fold}.pt'))

    test_dataset = IndoorWiFiDataSet(test_data, data_type='test')
    testloader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, num_workers=4)
    
    test_preds = []
    for i, (bssid, rssi, site_id) in tqdm(enumerate(testloader), total=len(testloader)):
        net.eval()

        with torch.no_grad():

            bssid, rssi, site_id = bssid.to(device).long(), rssi.to(device).float(), site_id.to(device).long()

            xy_pred, floor_pred = net(bssid, rssi, site_id)
            xy_pred = xy_pred.to('cpu').detach().numpy().copy()

            x_pred, y_pred = np.hsplit(xy_pred, 2)
            x_preds = np.concatenate([x_preds, x_pred.reshape(-1)])
            y_preds = np.concatenate([y_preds, y_pred.reshape(-1)])

    x_preds_mean += x_preds / Config.N_FOLD
    y_preds_mean += y_preds / Config.N_FOLD

/content/drive/MyDrive/ml/Indoor/output/load checkpoint_exp003-0_0.pt


100%|██████████| 10/10 [00:00<00:00, 41.21it/s]

/content/drive/MyDrive/ml/Indoor/output/load checkpoint_exp003-0_1.pt



100%|██████████| 10/10 [00:00<00:00, 43.02it/s]

/content/drive/MyDrive/ml/Indoor/output/load checkpoint_exp003-0_2.pt


...

/content/drive/MyDrive/ml/Indoor/output/load checkpoint_exp003-0_3.pt



100%|██████████| 10/10 [00:00<00:00, 43.21it/s]

/content/drive/MyDrive/ml/Indoor/output/load checkpoint_exp003-0_4.pt



100%|██████████| 10/10 [00:00<00:00, 42.18it/s]

CPU times: user 647 ms, sys: 1.99 s, total: 2.64 s
Wall time: 3.82 s


In [26]:
oof_score = comp_metric(oof_x, oof_y, np.zeros(len(data)), data['x'], data['y'], np.zeros(len(data)))
oof_score

5.733015777576308

In [27]:
np.save(ROOT_DIR / Path('output') / f'oof_x_{NB}',oof_x)
np.save(ROOT_DIR / Path('output') / f'oof_y_{NB}',oof_y)

In [28]:
sample_submission = pd.read_csv(DATA_DIR / 'sample_submission.csv')
submission_99per_acc = pd.read_csv(DATA_DIR / 'submission_99per_acc_.csv')

pred_df = test_data[['site_path_timestamp']]
pred_df['x'] = x_preds_mean
pred_df['y'] = y_preds_mean
sub = pred_df.merge(submission_99per_acc[['site_path_timestamp', 'floor']], how='left', on='site_path_timestamp')[['site_path_timestamp', 'floor', 'x', 'y']]

In [29]:
assert len(sample_submission.merge(sub, how='inner', on='site_path_timestamp')) == len(sample_submission)

In [30]:
sub_file_path = str(ROOT_DIR / Path('output') / f'sub_{NB}.csv')
sub.to_csv(sub_file_path, index=False)

## log mlflow 

In [ ]:
"""param_dict = {k:v for k, v in vars(Config).items() if not k.startswith('__')}

remote_server_uri = "http://192.168.1.99:5000/"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://192.168.1.99:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio-access-key"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio-secret-key"

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment("atma10")
with mlflow.start_run():
    mlflow.set_tag('NB', NB)
    mlflow.set_tag('DESCRIPTION', DESCRIPTION)
    mlflow.set_tag('HOST', HOST)
    
    mlflow.log_param("a", 1)
    mlflow.log_param('features', features)
    mlflow.log_params(param_dict)
    mlflow.log_params(params)
    
    mlflow.log_metric('oof', oof_score)
    mlflow.log_metrics(cv_scores)
    
    mlflow.log_artifact(get_notebook_path())
    mlflow.log_artifact(oof_file_name + '.npy')"""